In [ ]:
#Run the following code to print multiple outputs from a cell
get_ipython().ast_node_interactivity = 'all'

# Feature Engineering

## Importing & Profiling the Data
Import the file, "06_StockData.csv", and save it in a variable called `df`. Then, do some initial data profiling of the data based on what we covered the last 2 classes. 

What are the data types?

Are there missing values?

What are the summary statistics for the quantitative variables? What can you say about the variables based on the summary?

Create histograms for "stockShares", "stockPrice", and "temperature". Try using the parameter `bins=` to set the number of bins for your histogram (e.g., `bins=30`, `bins=40`, etc.).

In [ ]:
# Histogram for stockShares:


In [ ]:
# Histogram for stockPrice:


In [ ]:
# Histogram for temperature:


Can you make any conclusions about these variables from the histograms?

How many categories are there for "ticker symbol" and what are the categories? Do you notice anything strange about this variable?

## Renaming a Column

That space in the column name "ticker symbol" is really going to get annoying. Let's change it:

In [ ]:
df.rename(columns = {"ticker symbol" : "tickerSymbol"})

Quick python syntax side note: the command above includes a format that we've not seen before... `{"ticker symbol" : "ticker_symbol"}`. The braces (`{}`) define a *dictionary* in python, which associates one value with another. In this case, we are associating the old column name to the new column name.

Now check the data types again:

In [ ]:
df.dtypes

The column reverted back to the original name. Why?

Re-run the `.rename()` command below, this time saving your results and check the data types to make sure the change took effect:

## Adding new columns

Remember how we created variables in class 01? We used an assignment statement, like the following:

In [ ]:
x = 42

You do the same thing to add a new column to your DataFrame...the variable in this case being the name of the new column:

In [ ]:
df["Junk"] = 28
df

Now, you try it. Add a new column called "marketValue" that is calculated as "stockShares" * "stockPrice"

Then look at `df` to ensure the change took effect.

## Removing columns

Let's now get rid of that "Junk" column. To do this, use the `.drop()` method:

In [ ]:
df = df.drop(columns = "Junk")
df

## Changing Data

Remember the `tickerSymbol` column has a mix of upper- and lower-case ticker symbols:

In [ ]:
df["tickerSymbol"].value_counts().sort_index()

In python, you can change the case of a string using either `.str.upper()` or `.str.lower()`. Notice what happens when adding `.str.upper()` before the `.value_counts()` command:

In [ ]:
df["tickerSymbol"].str.upper().value_counts().sort_index()

Of course, that didn't change the actual values for that column:

In [ ]:
df["tickerSymbol"]

Using an assignment statement, change the "tickerSymbol" values to be all in upper-case and re-run the code in the previous cell to check that the changes worked:

### Using `.loc[]` to Change Data

What if there are only specific values we want to change?

Let's take another look at the distribution of the price data:

In [ ]:
df["stockPrice"].describe()
df["stockPrice"].plot.hist(bins = 40)

What are the clues that we have one or more outliers?

Once we've identified a value (such as a potential outlier) to change, how do we change it?

First, lets locate the row(s) with outliers based on some logical condition. We do this with `.loc[]`. 

In [ ]:
# Locating rows based on logical conditions:
df.loc[df["tickerSymbol"] == "AAPL"]

# Locating rows and column:
df.loc[df["tickerSymbol"] == "AAPL", "revenue"]

Remember, the general structure of the `.loc[]` command is:
```
dataFrame.loc[rowCriteria, columnCriteria]
```

Now, you try...locate any row with a "stockPrice" greater than 10000 and display only the "stockPrice" column:

Now that we have located the actual outlier value, we can change it with an assignment statement. Run the code below to change it to 10001:

In [ ]:
df.loc[df["stockPrice"] > 10000, "stockPrice"] = 10001
df.loc[df["stockPrice"] > 10000, "stockPrice"]

Of course, we don't want to make the outlier value even worse. Since it's likely a value greater than 10000 is an error, let's change it to `NaN` so that it doesn't impact our analysis. We can do this with `.nan` from the `numpy` module. 

Import `numpy` and change the outlier value to `NaN` as follows:

In [ ]:
import numpy as np
df.loc[df["stockPrice"] > 10000, "stockPrice"] = np.nan

Notice there is now a missing value for "stockPrice":

In [ ]:
df.isna().sum()

Run the following code to see the impact this has on our summary stats and histogram:

In [ ]:
df["stockPrice"].describe()
df["stockPrice"].plot.hist(bins = 40)

We're not done yet. Remember that python is sequential and we created the "marketValue" column *before* changing the outlier value. Here, we can see that the "marketValue" didn't change for this row:

In [ ]:
# I'm using df.index to refer to the specific row number from the DataFrame:
df.loc[df.index == 3446]

Re-run the following code cell to re-calculate "marketValue" and notice that we now have a missing value for that column (because python can't do math with a missing value):

In [ ]:
df["marketValue"] = df["stockShares"] * df["stockPrice"]
df.loc[df.index == 3446]
df.isna().sum()

## Handling Dates

Notice the "date" column has been read in as an "object", which means that python will treat it as a categorical variable. See if you can figure out how many unique dates there are in this data frame:

Usually we don't put date variables directly into our models because it would require n-1 dummy variables to handle each unique date "category". Instead, it's more useful to create a variable summarizing the dates, e.g., "weekday" or "month".

However, to do this we first must change the data type to "datetime." We can do this using the `.to_datetime()` function in `pandas`.

In [ ]:
pd.to_datetime(df["date"])

This just displays the dates as a "datetime" data type. Now, create a new variable called "date2" that changes "date" into a "datetime" object:

Notice, we didn't write over the original "date" variable in case we want to use the original data in the future.

Now, we can use datetime functions to summarize the dates. Run the code cell below to create a new column called "weekday":

In [ ]:
df["weekday"] = df["date2"].dt.day_name()

Now see if you can figure out how to modify the above code to create a new variable called "month" with the date's month:

## Transforming Features

### Squaring Variables
Plot temperature (x-axis) vs. revenue (y-axis):

Now, create a new column called "temperature2" which squares the temperature. Then, re-run the scatterplot using the new column:

***Squaring the variable is a good transformation technique when the original data shows a U-shaped curve.***

### Logging Variables
Now plot quantity (x-axis) vs. cogs (y-axis):

Here are the histograms for quantity and cogs:

In [ ]:
df["quantity"].plot.hist(bins = 40)

In [ ]:
df["cogs"].plot.hist(bins = 40)

Notice that cogs is relatively symmetric while quantity is quite skewed. Very often, a skew in the data can lead to the type of logarithmic curve we see in the scatter plot above.

Create a new column called "quantityLog" that logs the data in that column. You can use the `log()` function from `numpy` (which we've already imported as `np`). Then, re-run the scatterplot with the new column:

***Logging the variable is a good transformation technique when the original data shows a logarithmic curve (similar to half the U-shaped curve, although continuously increasing or decreasing). This type of curve often occurs if one of the variables is heavily skewed.***

## Interaction Features

Transformations can include more than one variable. For example: 

* Stock price alone tells us little about a company's performance. Typically we think of a price to earnings ratio
* Or we think about market cap, which is price times outstanding shares
* Absolute revenue is important, but the ratio of revenue to expenses might tell us more about efficiency

Let's look at the relationship of both revenue and cogs to market value.

First, plot revenue (x-axis) vs. marketValue (y-axis):

Now, plot cogs (x-axis) vs. marketValue (y-axis):

Now, let's take a look at how the ratio of revenue to expenses impacts market value. First, create a new column called "cogsRatio" which is calculated as revenue divided by cogs. Then create a scatterplot of ratio (x-axis) to marketValue (y-axis):

***Interacting 2 variables (e.g., multiplying them together or dividing them) will often create a stronger model feature than keeping the variables separate.*** Unfortunately, there is no obvious sign from the plots that you should interact 2 variables. Typically, interaction features are based on theory (or even just a hunch) that 2 variables interact with each other as they relate to a 3rd variable. If you suspect this is the case, try interacting the variables to see what happens.

## Creating New Features

### Transforming Unstructured Data
Unstructured data like text, image, audio, video, etc. require processing before we can use it in a model. 

You can get the length of a string by adding `.str.len()` to the variable as follows:

In [ ]:
df["statement"].str.len()

Add a column "statementLength" that is the length of the text in the statement and then plot statementLength (x-axis) vs. marketValue (y-axis):

### Transforming Numeric Data into Categorical Data

Sometimes, you may want to categorize your numeric data into groups based on some logical condition. For example, let's say that you wanted to see how the price of stocks impacts the number of stock shares. Create a variable called "highPrice" that is `True` if the stockPrice is greater than the median and `False` otherwise. 

Hint: to get `True`/`False`, you'll need a logical statement and you can use `.median()` in your statement.

Now create a side-by-side boxplot showing stockShares split by highPrice:

What if you wanted to create multiple categories? You can use the `select()` function from `numpy` as follows:

In [ ]:
conditions = [(df["stockPrice"] < 1003),   # < Q1 
              (df["stockPrice"] < 1433),   # < Q3
              (df["stockPrice"] >= 1433)]  # >= Q3

labels = ["low", "mid", "high"]

df["priceCat"] = np.select(conditions, labels, default = "unknown")
df["priceCat"].value_counts()

FYI, the 1 value of "unknown" is due to the fact that there was a missing value for the stockPrice column. You could use `np.nan` to change that "unknown" to a missing value as shown earlier in this worksheet.